In [42]:
'''images=[]
labels=[]

# loop over top 10 folders
for directory in os.listdir('/kaggle/input/leapgestrecog/leapGestRecog/'):
    for subdir in os.listdir(os.path.join('/kaggle/input/leapgestrecog/leapGestRecog/',directory)):
#         if directory=='01':
#             labels.append(subdir)
        for image in os.listdir(os.path.join('/kaggle/input/leapgestrecog/leapGestRecog/',directory, subdir)):
            img_path = os.path.join('/kaggle/input/leapgestrecog/leapGestRecog/', directory, subdir, image)
            images.append(img_path)
            labels.append(subdir)'''


"images=[]\nlabels=[]\n\n# loop over top 10 folders\nfor directory in os.listdir('/kaggle/input/leapgestrecog/leapGestRecog/'):\n    for subdir in os.listdir(os.path.join('/kaggle/input/leapgestrecog/leapGestRecog/',directory)):\n#         if directory=='01':\n#             labels.append(subdir)\n        for image in os.listdir(os.path.join('/kaggle/input/leapgestrecog/leapGestRecog/',directory, subdir)):\n            img_path = os.path.join('/kaggle/input/leapgestrecog/leapGestRecog/', directory, subdir, image)\n            images.append(img_path)\n            labels.append(subdir)"

In [43]:
import os 

training_images = []
training_labels = []
testing_images = []
testing_labels = []

# loop over top 10 folders
for directory in os.listdir('/kaggle/input/leapgestrecog/leapGestRecog/'):
    for subdir in os.listdir(os.path.join('/kaggle/input/leapgestrecog/leapGestRecog/', directory)):
        for image in os.listdir(os.path.join('/kaggle/input/leapgestrecog/leapGestRecog/', directory, subdir)):
            img_path = os.path.join('/kaggle/input/leapgestrecog/leapGestRecog/', directory, subdir, image)
            
            # Split into training and testing 
            if int(directory) <= 7:
                training_images.append(img_path)
                training_labels.append(subdir)
            else:
                testing_images.append(img_path)
                testing_labels.append(subdir)

print("Number of training images:", len(training_images))
print("Number of training labels:", len(training_labels))
print("Number of testing images:", len(testing_images))
print("Number of testing labels:", len(testing_labels))

Number of training images: 16000
Number of training labels: 16000
Number of testing images: 4000
Number of testing labels: 4000


In [44]:
label_to_integer = dict(zip(sorted(list(set(training_labels))), range(len(training_labels))))
label_to_integer

{'01_palm': 0,
 '02_l': 1,
 '03_fist': 2,
 '04_fist_moved': 3,
 '05_thumb': 4,
 '06_index': 5,
 '07_ok': 6,
 '08_palm_moved': 7,
 '09_c': 8,
 '10_down': 9}

In [45]:
training_int_labels = list(map(lambda x: label_to_integer[x], training_labels))
testing_int_labels = list(map(lambda x: label_to_integer[x], testing_labels))


In [46]:
import torch
from torch.utils.data import DataLoader
import glob
import os

# Prepare end indices for samplers
train_end_idx = []
test_end_idx = []


In [47]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #for different plot
from sklearn.preprocessing import OneHotEncoder #for data preprocessing
from sklearn.preprocessing import StandardScaler #for scaling data
from sklearn.model_selection import train_test_split
#for data preprocessing using keras
from keras.preprocessing.image import array_to_img, img_to_array, load_img
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# keras classes required for building deep CNN model
from keras.models import Sequential, save_model, load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Activation, Dropout 
from keras.utils import plot_model
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff
import gc

In [48]:
# Group images by subdirectory (gesture) and count
for directory in range(10):
    dir_path = f'/kaggle/input/leapgestrecog/leapGestRecog/{directory:02d}'
    for subdir in os.listdir(dir_path):
        subdir_path = os.path.join(dir_path, subdir)
        if os.path.isdir(subdir_path):
            paths = glob.glob(os.path.join(subdir_path, '*.png'))
            if directory <= 7:
                train_end_idx.append(len(paths))
            else:
                test_end_idx.append(len(paths))

In [49]:
# this code block divides into starting and ending of a specific gesture 
#converted into tensors 

train_end_idx = [0] + train_end_idx
train_end_idx = torch.cumsum(torch.tensor(train_end_idx), 0)

test_end_idx = [0] + test_end_idx
test_end_idx = torch.cumsum(torch.tensor(test_end_idx), 0)


In [50]:
#sequence of images 
seq_length = 20


In [51]:
test_end_idx = [0, *test_end_idx]
test_end_idx = torch.cumsum(torch.tensor(test_end_idx), 0)

In [52]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class MySampler(torch.utils.data.Sampler):
    def __init__(self, end_idx, seq_length):
        indices = []
        for i in range(len(end_idx)-1):
            start = end_idx[i]
            # end = end_idx[i+1] - seq_length
            end = end_idx[i+1] - seq_length
            if (end != -20 and end <= 15999):
                indices.append(torch.arange(start, end))
        indices = torch.cat(indices)

        self.indices = indices

    def __iter__(self):
        indices = self.indices[torch.randperm(len(self.indices))]
        return iter(indices.tolist())

    def __len__(self):
        return len(self.indices)


class MyDataset(Dataset):
    def __init__(self, image_paths, labels, seq_length, transform, length, device=("cuda" if torch.cuda.is_available() else "cpu")):
        self.image_paths = image_paths
        self.labels = labels
        self.seq_length = seq_length
        self.transform = transform
        self.length = length
        self.device = device
        self.n_frames = 100


    def __getitem__(self, index):
        start = index
        # end = index + self.seq_length
        end = index + self.n_frames
        # print('Getting images from {} to {}'.format(start, end))
        indices = list(range(start, end))
        images = []
        #print(start, end)
        #print(len(self.image_paths))
        for i in indices:
            image_path = self.image_paths[i]
            # print(image_path)
            # print('image_path :', image_path)
            image = Image.open(image_path)
            image = image.convert("RGB")
            if self.transform:
                image = self.transform(image)
            images.append(image)
        # print('=============================')

        #getting the video ID
        video_id = self.image_paths[i][0]
        vid = video_id.split('/')[-2]

        x = torch.stack(images).to(self.device)

        y = self.labels[index]
        return x, y

    def __len__(self):
        return self.length

In [53]:
# samplers creation here
import torchvision.transforms as transforms

train_sampler = MySampler(train_end_idx, 20)
test_sampler = MySampler(test_end_idx, seq_length)

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.225])
])


# defining the datset again in here
train_data = MyDataset(image_paths=training_images,
                       labels=training_int_labels,
                       seq_length=seq_length,
                       transform = transform,
                       length=len(train_sampler))

test_data = MyDataset(image_paths=testing_images,
                      labels=testing_int_labels,
                      seq_length=seq_length,
                      transform = transform,
                      length=len(test_sampler))

In [54]:
batch_size = 10
train_loader = DataLoader(dataset= train_data, batch_size=batch_size,sampler=train_sampler)
test_loader = DataLoader(dataset= test_data, batch_size=batch_size, sampler=test_sampler)


In [55]:
#include both image path and labels into dataframe
#df = pd.DataFrame({'Image':images,'Label':labels})
#df.tail()
# df.info()

In [56]:
'''unique_labels = np.unique(df['Label'])
label_to_integer = dict(zip(unique_labels, range(len(unique_labels))))

df['LabelInt'] = df['Label'].map(label_to_integer)'''

"unique_labels = np.unique(df['Label'])\nlabel_to_integer = dict(zip(unique_labels, range(len(unique_labels))))\n\ndf['LabelInt'] = df['Label'].map(label_to_integer)"

In [57]:
'''df['Label'] = df['LabelInt']'''

"df['Label'] = df['LabelInt']"

In [58]:
'''df = df.drop(columns=["LabelInt"])
df'''

'df = df.drop(columns=["LabelInt"])\ndf'

In [59]:
# from torch.utils.data import Dataset, DataLoader, ConcatDataset
# from PIL import Image
# from torchvision import datasets, models, transforms
# import torchvision
# import numpy as np
# class CNNDataset(Dataset):
#     """
#     Custom dataset definition
#     """
#     def __init__(self, df, mode='train', transform=None):
#         """
#         """
#         self.df = df
#         self.mode= mode
#         self.transform = transform
        
            
#     def __getitem__(self, index):
#         """
#         """
#         img_name = self.df.iloc[index]["Image"]
#         image = Image.open(img_name)
                
#         if self.mode == 'train':
#             image = image.convert("RGB")
                
#             image = np.asarray(image)
#             if self.transform is not None:
#                 image = self.transform(image)
#             labels =  np.asarray(self.df.iloc[index]["Label"]) #might not need this
#             return image, labels
            
#         elif self.mode == 'val':
#             image = image.convert("RGB")
            
#             image = np.asarray(image)
#             if self.transform is not None:
#                 image = self.transform(image)
#             labels = np.asarray(self.df.iloc[index]["Label"])
#             return image, labels
        
#     def __len__(self):
#         return len(self.df)

In [60]:
''' def get_dataloaders(input_size, batch_size, augment=False, shuffle = True):
    # How to transform the image when you are loading them.
    # you'll likely want to mess with the transforms on the training set.
    
    # For now, we resize/crop the image to the correct input size for our network,
    # then convert it to a [C,H,W] tensor, then normalize it to values with a given mean/stdev. These normalization constants
    # are derived from aggregating lots of data and happen to produce better results.
    data_transforms = {
        'train': transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.225])
        ]),
        'val': transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.225])
        ])
    }
    
    data_subsets = {x: CNNDataset(df, 
                                  mode = x,
                                  transform=data_transforms[x]) for x in data_transforms.keys()}
    # Create training and validation dataloaders
    # Never shuffle the test set
    dataloaders_dict = {x: DataLoader(data_subsets[x], batch_size=batch_size, shuffle=False if x != 'train' else shuffle, num_workers=4) for x in data_transforms.keys()}
    return dataloaders_dict
 '''

" def get_dataloaders(input_size, batch_size, augment=False, shuffle = True):\n    # How to transform the image when you are loading them.\n    # you'll likely want to mess with the transforms on the training set.\n    \n    # For now, we resize/crop the image to the correct input size for our network,\n    # then convert it to a [C,H,W] tensor, then normalize it to values with a given mean/stdev. These normalization constants\n    # are derived from aggregating lots of data and happen to produce better results.\n    data_transforms = {\n        'train': transforms.Compose([\n            transforms.ToPILImage(),\n            transforms.Resize(input_size),\n            transforms.CenterCrop(input_size),\n            transforms.ToTensor(),\n            transforms.Normalize([0.5], [0.225])\n        ]),\n        'val': transforms.Compose([\n            transforms.ToPILImage(),\n            transforms.Resize(input_size),\n            transforms.CenterCrop(input_size),\n            transform

In [61]:
'''import torch
import torchvision.models as models

# Load the pre-trained ResNet-18 model
model = models.vgg16(pretrained=True)

# Freeze all the pre-trained layers
for param in model.parameters():
    param.requires_grad = False


# Modify the last layer of the model
num_classes = 10 # replace with the number of classes in your dataset

#lastlayer = model.classifier[6]
#num_ftrs = model.fc.in_features
num_ftrs = model.classifier[6].in_features
#model.fc = torch.nn.Linear(num_ftrs, num_classes)
model.classifier[6] = torch.nn.Linear(num_ftrs, num_classes)'''

'import torch\nimport torchvision.models as models\n\n# Load the pre-trained ResNet-18 model\nmodel = models.vgg16(pretrained=True)\n\n# Freeze all the pre-trained layers\nfor param in model.parameters():\n    param.requires_grad = False\n\n\n# Modify the last layer of the model\nnum_classes = 10 # replace with the number of classes in your dataset\n\n#lastlayer = model.classifier[6]\n#num_ftrs = model.fc.in_features\nnum_ftrs = model.classifier[6].in_features\n#model.fc = torch.nn.Linear(num_ftrs, num_classes)\nmodel.classifier[6] = torch.nn.Linear(num_ftrs, num_classes)'

In [62]:
criterion = torch.nn.CrossEntropyLoss()

'''# Define the loss function and optimizer
#optimizer = torch.optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.SGD(model.classifier[6].parameters(), lr=0.001, momentum=0.9)'''

'# Define the loss function and optimizer\n#optimizer = torch.optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)\noptimizer = torch.optim.SGD(model.classifier[6].parameters(), lr=0.001, momentum=0.9)'

In [63]:
'''dataloaders = get_dataloaders(input_size=256, batch_size=64, shuffle=True)'''


'dataloaders = get_dataloaders(input_size=256, batch_size=64, shuffle=True)'

In [64]:
'''train_loader = dataloaders['train']
val_loader = dataloaders['val']'''

"train_loader = dataloaders['train']\nval_loader = dataloaders['val']"

In [65]:
import torch
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F

class CNNLSTM(nn.Module):
    def __init__(self, num_classes=10):
        super(CNNLSTM, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        self.resnet.fc = nn.Sequential(nn.Linear(self.resnet.fc.in_features, 300))
        self.lstm = nn.LSTM(input_size=300, hidden_size=256, num_layers=3)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, num_classes)
    def forward(self, x_3d):
        hidden = None
        # Iterate over each frame of a video in a video of batch * frames * channels * height * width
        for t in range(x_3d.size(1)):
            with torch.no_grad():
                x = self.resnet(x_3d[:, t])
            # Pass latent representation of frame through lstm and update hidden state
            out, hidden = self.lstm(x.unsqueeze(0), hidden)
        # Get the last hidden state (hidden is a tuple with both hidden and cell state in it)
        x = self.fc1(hidden[0][-1])
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [66]:
#shifting model to gpu
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#model.to(device)

In [67]:
num_classes = 10

#Different model parameters to play around with
num_epochs = 4
learning_rate = 0.01

In [68]:
def train():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model=CNNLSTM().to(device) #for the CNN LSTM
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    total_step = len(train_loader)
    
    #take out the following code if just starting to train
    #checkpoint = torch.load('PREVIOUS MODEL PATH')  for ex: torch.load('../input/trained-model/model_21.pth')
    #model.load_state_dict(checkpoint['state_dict'])
    #optimizer.load_state_dict(checkpoint['optimizer'])
    #epoch_before = checkpoint['epoch']
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(tqdm(train_loader), 1):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            
            outputs = model(inputs)
            outputs = torch.squeeze(outputs)
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        #'epoch' = epoch if just starting to train
        #'epoch' = epoch+epoch_before+1 afterwards
        model_file = { 'epoch': epoch,
                      'state_dict': model.state_dict(),
                      'optimizer' : optimizer.state_dict()}

        torch.save(model_file, "model" + str(epoch) + '.pth')  
        #str(epoch) if just starting to train 
        #str(epoch+epoch_before+1) afterwards

        model.eval()

        train_correct = 0
        train_total = 0
        with torch.no_grad():
            for data in tqdm(train_loader):
                images, labels = data
                
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)

                train_total += labels.size(0)

                train_correct += (predicted == labels).sum().item()
        print(train_correct, train_total)
                
        #str(epoch) if just starting to train
        #str(epoch+epoch_before+1) afterwards
        print("epoch: " + str(epoch))
        print('Top One Error of the network on train images: %d %%' % (
                100 * (1 - float(train_correct) / train_total)))


        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for data in tqdm(val_loader):
                images, labels = data

                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)

                val_total += labels.size(0)

                val_correct += (predicted == labels).sum().item()
        print(val_correct, val_total)

        #str(epoch) if just starting to train
        #str(epoch+epoch_before+1) afterwards
        print("epoch: " + str(epoch))
        print('Top One Error of the network on validation images: %d %%' % (
                100 * (1 - float(val_correct) / val_total)))
        

        gc.collect()

In [69]:
'''def train():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model=CNNLSTM().to(device) #for the CNN LSTM
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    total_step = len(train_loader)
    
    #take out the following code if just starting to train
    #checkpoint = torch.load('PREVIOUS MODEL PATH')  for ex: torch.load('../input/trained-model/model_21.pth')
    #model.load_state_dict(checkpoint['state_dict'])
    #optimizer.load_state_dict(checkpoint['optimizer'])
    #epoch_before = checkpoint['epoch']
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(tqdm(train_loader), 1):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            
            outputs = model(inputs)
            outputs = torch.squeeze(outputs)
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        #'epoch' = epoch if just starting to train
        #'epoch' = epoch+epoch_before+1 afterwards
        model_file = { 'epoch': epoch,
                      'state_dict': model.state_dict(),
                      'optimizer' : optimizer.state_dict()}

        torch.save(model_file, "model" + str(epoch) + '.pth')  
        #str(epoch) if just starting to train 
        #str(epoch+epoch_before+1) afterwards

        model.eval()

        train_correct = 0
        train_total = 0
        with torch.no_grad():
            for data in tqdm(train_loader):
                images, labels = data
                
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)

                train_total += labels.size(0)

                train_correct += (predicted == labels).sum().item()
        print(train_correct, train_total)
                
        #str(epoch) if just starting to train
        #str(epoch+epoch_before+1) afterwards
        print("epoch: " + str(epoch))
        print('Top One Error of the network on train images: %d %%' % (
                100 * (1 - float(train_correct) / train_total)))


        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for data in tqdm(val_loader):
                images, labels = data

                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)

                val_total += labels.size(0)

                val_correct += (predicted == labels).sum().item()
        print(val_correct, val_total)

        #str(epoch) if just starting to train
        #str(epoch+epoch_before+1) afterwards
        print("epoch: " + str(epoch))
        print('Top One Error of the network on validation images: %d %%' % (
                100 * (1 - float(val_correct) / val_total)))
        

        gc.collect()'''

'def train():\n    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")\n    model=CNNLSTM().to(device) #for the CNN LSTM\n    criterion = nn.CrossEntropyLoss()\n    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)\n\n    total_step = len(train_loader)\n    \n    #take out the following code if just starting to train\n    #checkpoint = torch.load(\'PREVIOUS MODEL PATH\')  for ex: torch.load(\'../input/trained-model/model_21.pth\')\n    #model.load_state_dict(checkpoint[\'state_dict\'])\n    #optimizer.load_state_dict(checkpoint[\'optimizer\'])\n    #epoch_before = checkpoint[\'epoch\']\n    \n    for epoch in range(num_epochs):\n        running_loss = 0.0\n        for i, (inputs, labels) in enumerate(tqdm(train_loader), 1):\n            inputs = inputs.to(device)\n            labels = labels.to(device)\n            optimizer.zero_grad()\n            \n            outputs = model(inputs)\n            outputs = torch.squeeze(outputs)\n            \n    

In [70]:
from tqdm import tqdm


In [ ]:
train()

  2%|▏         | 35/1440 [04:50<3:06:27,  7.96s/it]